<a href="https://colab.research.google.com/github/WittmannF/jupyter-translate/blob/master/playground.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Test `translate_markdown`

In [16]:
!pip install fire
!pip install googletrans

In [0]:
#export
import json, re
from googletrans import Translator

def translate_markdown(text, dest_language='pt'):
    # Regex expressions
    MD_CODE_REGEX='```[a-z]*\n[\s\S]*?\n```'
    CODE_REPLACEMENT_KW = 'xx_markdown_code_xx'

    MD_LINK_REGEX="\[[^)]+\)"
    LINK_REPLACEMENT_KW = 'xx_markdown_link_xx'
    
    # Markdown tags
    END_LINE='\n'
    IMG_PREFIX='!['
    HEADERS=['### ', '###', '## ', '##', '# ', '#'] # Should be from this order (bigger to smaller)

     # Inner function to replace tags from text from a source list        
    def replace_from_list(tag, text, replacement_list):
        list_to_gen = lambda: [(yield x) for x in replacement_list]
        replacement_gen = list_to_gen()
        return re.sub(tag, lambda x: next(replacement_gen), text) 

    # Create an instance of Tranlator
    translator = Translator()

    # Inner function for translation
    def translate(text):
        # Get all markdown links
        md_links = re.findall(MD_LINK_REGEX, text)

        # Get all markdown code blocks
        md_codes = re.findall(MD_CODE_REGEX, text)

        # Replace markdown links in text to markdown_link
        text = re.sub(MD_LINK_REGEX, LINK_REPLACEMENT_KW, text)

        # Replace links in markdown to tag markdown_link
        text = re.sub(MD_CODE_REGEX, CODE_REPLACEMENT_KW, text)

        # Translate text
        text = translator.translate(text, dest=dest_language).text

        # Replace tags to original link tags
        text = replace_from_list(LINK_REPLACEMENT_KW, text, md_links)
        
        # Replace code tags
        text = replace_from_list(CODE_REPLACEMENT_KW, text, md_codes)

        return text    

    # Check if there are special Markdown tags
    # TODO: Implent `code tag` and [link tag]() checker
    if len(text)>=2:
        if text[-1:]==END_LINE:
            return translate(text)+'\n'

        if text[:2]==IMG_PREFIX:
            return text

        for header in HEADERS:
            len_header=len(header)
            if text[:len_header]==header:
                return header + translate(text[len_header:])
        
    return translate(text)

In [18]:
def test_translate_markdown():

    markdown_with_link = """It's based on research in to deep learning best practices undertaken at [fast.ai](http://www.fast.ai), including "out of the box" support for [`vision`](/vision.html#vision), [`text`](/text.html#text), [`tabular`](/tabular.html#tabular), and [`collab`](/collab.html#collab)(collaborative filtering) models. If you're looking for the source code, head over to the [fastai repo](https://github.com/fastai/fastai) on GitHub. For brief examples, see the [examples](https://github.com/fastai/fastai/tree/master/examples) folder; detailed examples are provided in the full documentation (see the sidebar). For example, here's how to train an MNIST model using [resnet18](https://arxiv.org/abs/1512.03385) (from the [vision example](https://github.com/fastai/fastai/blob/master/examples/vision.ipynb)):"""

    markdown_with_code = """In general, for interactive computing, to just play around the core modules and the training loop you can do

```
from fastai.vision import *
from fastai.core import *
from fastai.basics import *
```

"""

    texts = ['### Hello',  '# Hello', '![img_name](img.png)', '## Hello', '##Hey you', markdown_with_link, markdown_with_code]

    for t in texts:
        print(translate_markdown(t))

test_translate_markdown()

### Olá
# Olá
![img_name](img.png)
## Olá
##ei você
É baseado em pesquisa para profundas melhores práticas de aprendizagem empreendidas a [fast.ai](http://www.fast.ai), incluindo "fora da caixa" suporte para [`vision`](/vision.html#vision), [`text`](/text.html#text), [`tabular`](/tabular.html#tabular) e [`collab`](/collab.html#collab) (filtragem colaborativa) modelos. Se você está procurando o código fonte, de cabeça para o [fastai repo](https://github.com/fastai/fastai) no GitHub. Por breves exemplos, ver a pasta [examples](https://github.com/fastai/fastai/tree/master/examples); exemplos detalhados são fornecidos na documentação completa (veja o quadro). Por exemplo, aqui está como treinar um modelo MNIST usando [resnet18](https://arxiv.org/abs/1512.03385) (do [vision example](https://github.com/fastai/fastai/blob/master/examples/vision.ipynb)):
Em geral, para computação interativa, jogar apenas em torno dos módulos principais eo loop treinamento que você pode fazer

```
from fastai.v

## Test Jupyter Translate

In [0]:
#export
def jupyter_translate_old(fname, language='pt', rename_source_file=False, print_translation=False):
    data_translated = json.load(open(fname, 'r'))

    for i in range(len(data_translated['cells'])):
        for j in range(len(data_translated['cells'][i]['source'])):
            if data_translated['cells'][i]['cell_type']=='markdown':
                data_translated['cells'][i]['source'][j] = \
                  translate_markdown(data_translated['cells'][i]['source'][j], dest_language=language)
            if print_translation:
                print(data_translated['cells'][i]['source'][j])

    if rename_source_file:
        fname_bk = f"{'.'.join(fname.split('.')[:-1])}_bk.ipynb" # index.ipynb -> index_bk.ipynb
        
        os.rename(fname, fname_bk)
        print(f'{fname} was renamed into {fname_bk}')
        
        open(fname,'w').write(json.dumps(data_translated))
        print(f'The {language} translation was stored in {fname}')
    else:
        dest_fname = f"{'.'.join(fname.split('.')[:-1])}_{language}.ipynb" # any.name.ipynb -> any.name_pt.ipynb
        open(dest_fname,'w').write(json.dumps(data_translated))
        print(f'The {language} translation was stored in {dest_fname}')

In [0]:
#export
def jupyter_translate(fname, language='pt', rename_source_file=False, print_translation=False):
    data_translated = json.load(open(fname, 'r'))
    
    skip_row=False
    for i, cell in enumerate(data_translated['cells']):
        for j, source in enumerate(cell['source']):
            if cell['cell_type']=='markdown':
                if source[:3]=='```':
                    skip_row = not skip_row # Invert flag until I find next code block

                if not skip_row:
                    if source not in ['```\n', '```', '\n'] and source[:4] != '<img':  # Don't translate cause
                    # of: 1. ``` -> ëëë 2. '\n' disappeared 3. image's links damaged
                        data_translated['cells'][i]['source'][j] = \
                            translate_markdown(source, dest_language=language)
            if print_translation:
                print(data_translated['cells'][i]['source'][j])

    if rename_source_file:
        fname_bk = f"{'.'.join(fname.split('.')[:-1])}_bk.ipynb" # index.ipynb -> index_bk.ipynb
        
        os.rename(fname, fname_bk)
        print(f'{fname} was renamed into {fname_bk}')
        
        open(fname,'w').write(json.dumps(data_translated))
        print(f'The {language} translation has been saved as {fname}')
    else:
        dest_fname = f"{'.'.join(fname.split('.')[:-1])}_{language}.ipynb" # any.name.ipynb -> any.name_pt.ipynb
        open(dest_fname,'w').write(json.dumps(data_translated))
        print(f'The {language} translation has been saved as {dest_fname}')

In [36]:
#!wget https://github.com/WittmannF/fastai/raw/master/docs_src/index.ipynb
jupyter_translate('index.ipynb', print_translation=True)

# Bem-vindo ao fastai
from fastai.vision import *

from fastai.gen_doc.nbdoc import *

from fastai.core import *

from fastai.basic_train import *
A biblioteca fastai simplifica o treinamento de redes neurais rápidos e precisos utilizando as melhores práticas modernas. É baseado em pesquisa para profundas melhores práticas de aprendizagem empreendidas a [fast.ai](http://www.fast.ai), incluindo "fora da caixa" suporte para [`vision`](/vision.html#vision), [`text`](/text.html#text), [`tabular`](/tabular.html#tabular) e [`collab`](/collab.html#collab) (filtragem colaborativa) modelos. Se você está procurando o código fonte, de cabeça para o [fastai repo](https://github.com/fastai/fastai) no GitHub. Por breves exemplos, ver a pasta [examples](https://github.com/fastai/fastai/tree/master/examples); exemplos detalhados são fornecidos na documentação completa (veja o quadro). Por exemplo, aqui está como treinar um modelo MNIST usando [resnet18](https://arxiv.org/abs/1512.03385) (do [vision ex

## Mount GDrive and investigate file

In [22]:
!pip install easycolab
import easycolab as ec
ec.mount()

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
Opening directory /content/gdrive/My Drive/
Done!


In [23]:
cd '_FASTAI'

/content/gdrive/My Drive/_FASTAI


In [24]:
ls

 course-v3/            fastai-fork/
 fastai-dlpt2-notes/   Lessons/
 fastai_docs/          new_class_material/
 fastai_docs_ptbr/    'Screen Shot 2019-04-10 at 18.01.23.png'


In [25]:
cd fastai-fork/

/content/gdrive/My Drive/_FASTAI/fastai-fork


In [26]:
cd docs_src/pt-br

/content/gdrive/My Drive/_FASTAI/fastai-fork/docs_src/pt-br


In [29]:
ls

 applications.ipynb              points.pth
 basic_data.ipynb                pytest.ini
 basic_train.ipynb               run_tests.sh
 callback.ipynb                  sidebar/
 callbacks.csv_logger.ipynb      tabular.data.ipynb
 callbacks.fp16.ipynb            tabular.ipynb
 callbacks.general_sched.ipynb   tabular.models.ipynb
 callbacks.hooks.ipynb           tabular.transform.ipynb
 callbacks.ipynb                 text.data.ipynb
 callbacks.lr_finder.ipynb       text.interpret.ipynb
 callbacks.mem.ipynb             text.ipynb
 callbacks.misc.ipynb            text.learner.ipynb
 callbacks.mixup.ipynb           text.models.ipynb
 callbacks.one_cycle.ipynb       text.transform.ipynb
 callbacks.rnn.ipynb             torch_core.ipynb
 callbacks.tensorboard.ipynb     training.ipynb
 callbacks.tracker.ipynb         train.ipynb
 collab.ipynb                    trustnbs.py
 conftest.py                     tutorial.data.ipynb
 core.ipynb                      tutorial.inference.ipynb
 data_block

In [0]:
#!wget https://raw.githubusercontent.com/WittmannF/jupyter-translate/master/jupyter_translate.py

Now let's install the requirements and import the `jupyter_translate` module

In [35]:
jupyter_translate('index.ipynb', print_translation=True, language='fr')

# Bienvenue sur fastai
from fastai.vision import *

from fastai.gen_doc.nbdoc import *

from fastai.core import *

from fastai.basic_train import *
La bibliothèque fastai simplifie la formation des réseaux neuronaux rapides et précises en utilisant les meilleures pratiques modernes. Il est basé sur la recherche dans les meilleures pratiques d'apprentissage profondes entreprises à [fast.ai](http://www.fast.ai), y compris « hors de la boîte » pour le soutien [`vision`](/vision.html#vision), [`text`](/text.html#text), [`tabular`](/tabular.html#tabular) et modèles [`collab`](/collab.html#collab) (filtrage collaboratif). Si vous cherchez le code source, la tête sur le [fastai repo](https://github.com/fastai/fastai) sur GitHub. Pour de brefs exemples, voir le dossier [examples](https://github.com/fastai/fastai/tree/master/examples); des exemples détaillés sont fournis dans la documentation complète (voir l'encadré). Par exemple, voici comment former un modèle en utilisant MNIST [resnet18](ht